# Домашнее задание 1

### Описание

В вашем распоряжении датасет с русскоязычными отзывами о мобильных телефонах с выставленным рейтингом от 1 до 5.
Ключевая задача – обучить любую модель регрессии (или классификации, если решите таким путём пойти) из пакетов scikit, XGBoost, LightGBM, CatBoost.


Необходимая метрика:

1. Со звёздочкой (дополнительный балл) – MAE <= 0.5
2. Минимальное допустимое значение – МАЕ <= 1.0

### Что необходимо сделать

1. Откройте датасет
2. Разделите на обучение и тест
3. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 
4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).
5. Вывести ТОП-50 наиболее частотных токенов:
- только для униграмм
- только для биграмм
- только для триграмм
- для всех вариантов n-грамм одновременно

Напишите, какие наблюдения и выводы есть.
6. Повторите пункт 5, только отдельно для отзывов с рейтингом «4-5», «3» и «1-2». Есть ли ключевые отличия? Есть кандидаты на попадание в список стоп-слов?

7. Составьте список своих ключевых слов, для помощи можно использовать nltk и punctuations из string. 
8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer (экспериментируйте с параметрами min_df  и max_df). 
9. Обучение одну или несколько моделей машинного обучения на разных представлениях данных
10. Валидируйте модель. Если модель соответствует условиям метрик, то работа завершена. В ином случае, экспериментируйте, начиная с пункта 7. 
11. По всем попыткам обучить качественную модель пишите свои выводы и замечания, почему так получилось.


## Комментарии исполнителя

Я изменил порядок 2 и 3 пункта, потому что так намного удобнее

## 0. Импорт библиотк, определение констант

In [1]:
import os
import pandas as pd
import numpy as np
import spacy
import re

from string import punctuation
from collections import Counter

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from pymorphy3 import MorphAnalyzer

from nltk import ngrams
from nltk.corpus import stopwords

from sklearn.linear_model import LinearRegression

from catboost import CatBoost, CatBoostRegressor

from sklearn.metrics import classification_report

In [2]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [3]:
tqdm.pandas()

In [4]:
#!python -m spacy download ru_core_news_sm

## 1. Откройте датасет

In [5]:
if os.path.exists("data/data_lemma_cleared.csv"):
    df = pd.read_csv("data/data_lemma_cleared.csv", engine='python')
elif os.path.exists("data/data_lemma.csv"):
    df = pd.read_csv("data/data_lemma.csv", engine='python')
else:    
    df = pd.read_csv("data/data.csv")

In [6]:
df.shape

(382873, 3)

In [7]:
df.head(10)

,Review,Rating,lemma
0,3d touch просто восхитительная вещь заряд дер...,5.0,3d touch просто восхитительный вещь заряд держ...
1,отключается при температуре близкой к нулю не...,4.0,отключаться температура близкий нуль непонятно...
2,в apple окончательно решили не заморачиваться ...,3.0,apple окончательно решить не заморачиваться де...
3,постарался наиболее ёмко и коротко описать все...,4.0,постараться наиболее ёмко коротко описать всё ...
4,достойный телефон пользоваться одно удовольст...,5.0,достойный телефон пользоваться удовольствие
5,6s gold 64gb,5.0,6s gold 64gb
6,мой первый айфон скажу честно эппл ранее ник...,5.0,первый айфон сказать честно эппл ранее рука не...
7,мне очень понравилась эта модель во первых кл...,5.0,очень понравиться модель первый классный дизай...
8,долгое время пользовалась iphone 5s 16gb он м...,5.0,долгий время пользоваться iphone 5s 16gb устра...
9,раньше был samsung galaxy alpha sm g850f 32gb ...,4.0,ранний samsung galaxy alpha sm g850f 32gb купи...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382873 entries, 0 to 382872
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Review  382873 non-null  object 
 1   Rating  319867 non-null  float64
 2   lemma   319791 non-null  object 
dtypes: float64(1), object(2)
memory usage: 8.8+ MB


In [9]:
df = df.dropna().reset_index(drop=True)
df.head()

,Review,Rating,lemma
0,3d touch просто восхитительная вещь заряд дер...,5.0,3d touch просто восхитительный вещь заряд держ...
1,отключается при температуре близкой к нулю не...,4.0,отключаться температура близкий нуль непонятно...
2,в apple окончательно решили не заморачиваться ...,3.0,apple окончательно решить не заморачиваться де...
3,постарался наиболее ёмко и коротко описать все...,4.0,постараться наиболее ёмко коротко описать всё ...
4,достойный телефон пользоваться одно удовольст...,5.0,достойный телефон пользоваться удовольствие


In [10]:
df['Rating'] = df['Rating'].astype('Int8')
df['Rating'].value_counts()

Rating
5    159406
4     75014
3     37095
2     24627
1     23649
Name: count, dtype: Int64

In [11]:
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

In [12]:
df['Rating'].value_counts(normalize=True)

Rating
5    0.498469
4    0.234572
3    0.115998
2     0.07701
1    0.073951
Name: proportion, dtype: Float64

Я закомментил все что ниже, потому что такую обработку лучше делать после лемматизации

In [13]:
df.duplicated().sum()

8

In [14]:
df[df.duplicated()].head(10)

,Review,Rating,lemma
4285,p s жаль наушники в комплекте больше с люмиям...,5,брать смартфон программа тест драйв microsoft ...
15372,у меня всё,3,сразу вооружаться защитный плёнка экран тонкий...
20243,в подарок или себе этот сенсорный телефон вас ...,5,целое телефон очень удачный радовать свой прор...
38235,это потому что google предоставит еще одну ос...,2,главный страница поддержка вопрос ответ почему...
42398,достоинства перекрывают незначительные недоста...,5,яблоко самсунг замылить глаз решить купить аль...
50713,изучив смартфоны в данной ценовой категории с...,5,здорово чёрный цвет серый батарея хватать 1 5 ...
58266,p s про вес аппарата по субъективным ощущен...,5,владеть данные телефон белый цвет момент начал...
60695,про питание: батарейка держит отлично два час...,5,претензия телефон идеальный не бывать плэя мар...


In [15]:
df["Review"].value_counts()

Review
рекомендую                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  36
со

Основная причина дублей - короткие одинаковые комментарии, такие тубли можно исключить

In [16]:
# df = df.drop_duplicates()

In [17]:
df["Review"].value_counts()

Review
рекомендую                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  36
со

In [18]:
df[df["Review"] == "-"].value_counts()

Series([], Name: count, dtype: int64)

Исключим записи с одинаковым комментарием, но выоским разрбросом оценки

In [19]:
#bad_comments = df["Review"].value_counts()
#bad_comments = bad_comments[bad_comments >= 3]
#bad_comments.head(20)

#bad_comments = bad_comments[bad_comments['value'].isin((4,5))]

In [20]:
df['Review'] = df['Review'].str.lower()

Уберём синтаксические знаки

In [21]:
punct = list(punctuation)
punct.remove(":")
punct.remove(")")
punct.remove("(")
punct = ''.join(punct)

In [22]:
df['Review']

0         3d touch просто восхитительная вещь  заряд дер...
1         отключается при температуре близкой к нулю  не...
2         в apple окончательно решили не заморачиваться ...
3         постарался наиболее ёмко и коротко описать все...
4         достойный телефон  пользоваться одно удовольст...
                                ...                        
319786    удобный  всё работает отлично  звонит  играет ...
319787    прошло больше года  притензий нет  при моей на...
319788    мой первый аппарат на андроиде  на данный моме...
319789    разбил iphone и не было желания покупать новый...
319790             очень доволен покупкой и всем советую   
Name: Review, Length: 319791, dtype: object

In [23]:
punct

'!"#$%&\'*+,-./;<=>?@[\\]^_`{|}~'

In [24]:
df['Review'] = df['Review'].str.replace(r'[' + re.escape(punct) + '\n]', ' ', regex=True)

## 2. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 

In [25]:
stop_words = set(stopwords.words('russian'))
stop_words.remove("не"); 

### spacy

In [26]:
spacy_nlp = spacy.load("ru_core_news_sm")

In [27]:
def spacy_lemmatize_text(text):
    try:
        doc = spacy_nlp(text)
    except:
        print(f'{text}')
        return ""
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

В этом месте у меня падала лемматизация, так я обнаружил что есть пустые комменты, но трай эксепт оставил, в пайморфек уже нет этого, потому что мусор удален

### pymorphy3

In [28]:
pymorphy3_analyzer = MorphAnalyzer()

In [29]:
def pymorphy3_lemmatize_text(text):
    lemmas = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    clean_tokens = [w for w in lemmas if not w in stop_words]  
    return (' '.join(clean_tokens))

In [30]:
df.head(50)

,Review,Rating,lemma
0,3d touch просто восхитительная вещь заряд дер...,5,3d touch просто восхитительный вещь заряд держ...
1,отключается при температуре близкой к нулю не...,4,отключаться температура близкий нуль непонятно...
2,в apple окончательно решили не заморачиваться ...,3,apple окончательно решить не заморачиваться де...
3,постарался наиболее ёмко и коротко описать все...,4,постараться наиболее ёмко коротко описать всё ...
4,достойный телефон пользоваться одно удовольст...,5,достойный телефон пользоваться удовольствие
5,6s gold 64gb,5,6s gold 64gb
6,мой первый айфон скажу честно эппл ранее ник...,5,первый айфон сказать честно эппл ранее рука не...
7,мне очень понравилась эта модель во первых кл...,5,очень понравиться модель первый классный дизай...
8,долгое время пользовалась iphone 5s 16gb он м...,5,долгий время пользоваться iphone 5s 16gb устра...
9,раньше был samsung galaxy alpha sm g850f 32gb ...,4,ранний samsung galaxy alpha sm g850f 32gb купи...


### Лемматинизируем

In [31]:
if not "lemma" in df.columns:
    print('Лемматизируем текст');
    df["lemma"] = df["Review"].progress_apply(lambda row: pymorphy3_lemmatize_text(row))
    df.to_csv("data/data_lemma_.csv", index=False)

In [32]:
df = df.dropna()
df = df.drop_duplicates()

In [33]:
df.shape

(319783, 3)

In [34]:
if not os.path.exists("data/data_lemma_cleared.csv"):
    df.to_csv("data/data_lemma_cleared.csv", index=False)

## 3. Разделите на обучение и тест

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df['lemma'], df['Rating'], test_size=0.2, stratify=df['Rating'])

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(255826,)
(255826,)
(63957,)
(63957,)


## 4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).

Я всё таки попробую разные способы, чтобы сравнить быстродействие и результат

In [36]:
def ngrams_stas(data, n):
    word_vectorizer = CountVectorizer(ngram_range=(n,n), analyzer='word')
    sparse_matrix = word_vectorizer.fit_transform(data)
    frequencies = sum(sparse_matrix).data
    return pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names_out(), columns=['frequency'])

### Униграммы

In [37]:
unigrams = list(df['lemma'].str.split()) 

In [38]:
unigrams = [x for xs in unigrams for x in xs]

In [39]:
len(unigrams)

14625387

In [40]:
print(len(set(unigrams)))

220501


In [41]:
unigrams

['3d',
 'touch',
 'просто',
 'восхитительный',
 'вещь',
 'заряд',
 'держать',
 'целый',
 'день',
 'розовый',
 'цвет',
 'смотреться',
 'очень',
 'необычно',
 'touch',
 'id',
 'очень',
 'быстрый',
 'удобный',
 'весь',
 'советовать',
 'телефон',
 'отключаться',
 'температура',
 'близкий',
 'нуль',
 'непонятно',
 'вести',
 'батарея',
 'apple',
 'окончательно',
 'решить',
 'не',
 'заморачиваться',
 'делать',
 'незначительный',
 'изменение',
 'телефон',
 'выдавать',
 'изменение',
 'инновация',
 'скопировать',
 'не',
 'функционал',
 'дизайн',
 'цена',
 'неадекватно',
 'завысить',
 'скачок',
 'курс',
 'говорить',
 'это',
 'не',
 'приходиться',
 'многие',
 'сказать',
 'hd',
 'разрешение',
 'достаточно',
 'диагональ',
 'мочь',
 'оно',
 'аппарат',
 'позиционироваться',
 'топовый',
 'решение',
 'итог',
 'получать',
 'банальный',
 'экономия',
 'производитель',
 'произойти',
 'время',
 'работа',
 'увеличить',
 'fhd',
 '1',
 'гб',
 'оперативка',
 'прошлый',
 'век',
 'не',
 'рассказывать',
 'сказка',


In [42]:
unigrams_counter = Counter(unigrams)
unigrams_counter.most_common(50)

[('не', 608793),
 ('телефон', 346915),
 ('это', 206961),
 ('всё', 164741),
 ('очень', 143984),
 ('хороший', 128714),
 ('экран', 90506),
 ('камера', 78017),
 ('аппарат', 77896),
 ('купить', 76123),
 ('работать', 72265),
 ('весь', 72021),
 ('год', 71850),
 ('день', 71603),
 ('2', 71488),
 ('свой', 61103),
 ('пользоваться', 59839),
 ('цена', 58861),
 ('просто', 56792),
 ('который', 56182),
 ('мочь', 55894),
 ('5', 55839),
 ('большой', 54543),
 ('смартфон', 53120),
 ('ещё', 52379),
 ('брать', 50267),
 ('модель', 49125),
 ('3', 48785),
 ('батарея', 47646),
 ('качество', 47049),
 ('деньга', 45184),
 ('приложение', 44833),
 ('довольный', 44539),
 ('отличный', 44279),
 ('4', 43531),
 ('проблема', 42727),
 ('время', 42307),
 ('покупать', 40353),
 ('покупка', 39456),
 ('рука', 36674),
 ('месяц', 36435),
 ('использование', 35899),
 ('первый', 35845),
 ('интернет', 34816),
 ('работа', 34528),
 ('1', 34018),
 ('память', 31379),
 ('хватать', 30849),
 ('стоить', 30706),
 ('целое', 30474)]

Попробуем создать нграммы через универсальную функцию с помощью CountVectorizer

In [43]:
#ngrams_stas(df['lemma'], 1).sort_values('frequency', ascending=False)

Получился странный и непонятный мне результат

### биграммы и триграммы
попробуем через nltk

In [44]:

un, bi, th = [], [], []
for s in df['lemma']:
    sent = s.split()
    un.append(list(ngrams(sent, 1)))
    bi.append(list(ngrams(sent, 2)))
    th.append(list(ngrams(sent, 3)))

Работаем мнгновенно!

In [45]:
un = [x for xs in un for x in xs]
un_counter = Counter(un)
un_counter.most_common(50)

[(('не',), 608793),
 (('телефон',), 346915),
 (('это',), 206961),
 (('всё',), 164741),
 (('очень',), 143984),
 (('хороший',), 128714),
 (('экран',), 90506),
 (('камера',), 78017),
 (('аппарат',), 77896),
 (('купить',), 76123),
 (('работать',), 72265),
 (('весь',), 72021),
 (('год',), 71850),
 (('день',), 71603),
 (('2',), 71488),
 (('свой',), 61103),
 (('пользоваться',), 59839),
 (('цена',), 58861),
 (('просто',), 56792),
 (('который',), 56182),
 (('мочь',), 55894),
 (('5',), 55839),
 (('большой',), 54543),
 (('смартфон',), 53120),
 (('ещё',), 52379),
 (('брать',), 50267),
 (('модель',), 49125),
 (('3',), 48785),
 (('батарея',), 47646),
 (('качество',), 47049),
 (('деньга',), 45184),
 (('приложение',), 44833),
 (('довольный',), 44539),
 (('отличный',), 44279),
 (('4',), 43531),
 (('проблема',), 42727),
 (('время',), 42307),
 (('покупать',), 40353),
 (('покупка',), 39456),
 (('рука',), 36674),
 (('месяц',), 36435),
 (('использование',), 35899),
 (('первый',), 35845),
 (('интернет',), 34

Совпадает с тем, что делал вручную и работает достаточно быстро. Делаем универсальную функцию

In [46]:
def ngrams_stats_nltk(data):
    
    un, bi, th = [], [], []
    
    for s in data:
        sent = s.split()
        un.append(list(ngrams(sent, 1)))
        bi.append(list(ngrams(sent, 2)))
        th.append(list(ngrams(sent, 3)))
        
    un = [x for xs in un for x in xs]
    un = Counter(un)
    
    bi = [x for xs in bi for x in xs]
    bi = Counter(bi)
    
    th = [x for xs in th for x in xs]
    th = Counter(th)       
    
    return pd.DataFrame({
                        'Униграмы' : un.most_common(50),
                        'Биграммы' : bi.most_common(50),
                        'Триграммы' : th.most_common(50)  
                        })
      
        

## 5. Топы

### Топ 50 для всех отзывов

In [47]:
ngrams_stats_nltk(df["lemma"])


,Униграмы,Биграммы,Триграммы
0,"((не,), 608793)","((телефон, не), 22864)","((соотношение, цена, качество), 4461)"
1,"((телефон,), 346915)","((это, не), 18715)","((брать, не, пожалеть), 3478)"
2,"((это,), 206961)","((не, мочь), 15252)","((стоить, свой, деньга), 3232)"
3,"((всё,), 164741)","((свой, деньга), 13652)","((игра, не, играть), 2074)"
4,"((очень,), 143984)","((очень, довольный), 13342)","((телефон, очень, довольный), 1967)"
5,"((хороший,), 128714)","((это, телефон), 12702)","((1, 5, год), 1942)"
6,"((экран,), 90506)","((не, пожалеть), 11323)","((телефон, свой, деньга), 1795)"
7,"((камера,), 78017)","((телефон, очень), 10551)","((очень, хороший, телефон), 1662)"
8,"((аппарат,), 77896)","((очень, хороший), 10368)","((2, 3, день), 1576)"
9,"((купить,), 76123)","((не, очень), 10101)","((телефон, очень, хороший), 1491)"


## 6. Топы по оценкам

### Топ 50 для плохих отзывов

In [48]:
ngrams_stats_nltk(df.loc[df["Rating"] < 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 101943)","((телефон, не), 4602)","((не, советовать, покупать), 909)"
1,"((телефон,), 56219)","((не, советовать), 4305)","((не, советовать, брать), 669)"
2,"((это,), 33940)","((это, не), 3091)","((никто, не, советовать), 589)"
3,"((всё,), 20075)","((не, мочь), 3066)","((большой, не, купить), 462)"
4,"((купить,), 15061)","((это, телефон), 2323)","((не, покупать, телефон), 415)"
5,"((очень,), 14287)","((не, покупать), 2308)","((стоить, свой, деньга), 378)"
6,"((хороший,), 12528)","((не, рекомендовать), 2270)","((жить, свой, жизнь), 373)"
7,"((экран,), 12457)","((не, стоить), 2017)","((не, стоить, свой), 362)"
8,"((год,), 11053)","((не, брать), 1954)","((1, 5, год), 319)"
9,"((работать,), 10217)","((не, работать), 1935)","((кой, случай, не), 290)"


In [49]:
ngrams_stats_nltk(df.loc[df["Rating"] == 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 75659)","((телефон, не), 3270)","((телефон, не, плохой), 343)"
1,"((телефон,), 41850)","((это, не), 2293)","((1, 5, год), 319)"
2,"((это,), 24792)","((не, мочь), 2117)","((стоить, свой, деньга), 296)"
3,"((всё,), 16944)","((это, телефон), 1646)","((не, советовать, покупать), 248)"
4,"((хороший,), 13615)","((не, очень), 1493)","((не, советовать, брать), 218)"
5,"((очень,), 12835)","((не, знать), 1355)","((оставлять, желать, хороший), 196)"
6,"((экран,), 10984)","((не, стоить), 1330)","((деньга, не, стоить), 185)"
7,"((год,), 9207)","((не, советовать), 1323)","((соотношение, цена, качество), 181)"
8,"((купить,), 9197)","((не, плохой), 1264)","((игра, не, играть), 181)"
9,"((камера,), 9112)","((свой, деньга), 1208)","((свой, деньга, не), 180)"


### Топ 50 для хороших отзывов

In [50]:
ngrams_stats_nltk(df.loc[df["Rating"] > 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 431191)","((телефон, не), 14992)","((соотношение, цена, качество), 4157)"
1,"((телефон,), 248846)","((это, не), 13331)","((брать, не, пожалеть), 3409)"
2,"((это,), 148229)","((очень, довольный), 12704)","((стоить, свой, деньга), 2558)"
3,"((всё,), 127722)","((свой, деньга), 11470)","((телефон, очень, довольный), 1917)"
4,"((очень,), 116862)","((не, пожалеть), 11002)","((игра, не, играть), 1791)"
5,"((хороший,), 102571)","((не, мочь), 10069)","((телефон, свой, деньга), 1622)"
6,"((экран,), 67065)","((очень, хороший), 9517)","((очень, хороший, телефон), 1594)"
7,"((камера,), 61223)","((телефон, очень), 9100)","((телефон, очень, хороший), 1388)"
8,"((аппарат,), 59529)","((это, телефон), 8733)","((2, 3, день), 1327)"
9,"((весь,), 56411)","((хороший, телефон), 8699)","((удобно, лежать, рука), 1322)"


Смотреть объединенную статистику бессмысленно, так как частотность униграм превышает частотность би и триграмм, соотвественном будет состоять из униграм.

### Вывод

Чем выше размер энграм , тем существенней отличия в положительных и отрицательных отзывах. По триграммам отлично понятно - положительный или отрицательный отзыв. Гипотеза - фичи на триграммах дадут лучший результат. 
Заметил присутствие слова "это", думаю никакой смысловой нагрузке в данном случае оно не несёт и его можно смело удалить. Так как и слова "телефон" , "сей", "пора", "смартфон" . Остальное оставлю, чтобы не портить значащие би и три граммы.

## 7. Список стоп слов

Базовый список стоп слов и пунктуацию мы использовали в предыдущиз шагах, в этом сделаем дополнительный список на основании анализа энграм

In [51]:
custom_stopwords = ['это', 'телефон', 'сей', 'пора', 'смартфон']

## 8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer 

(экспериментируйте с параметрами min_df  и max_df). 

В интернетах пишут
So to sum it up, pruning the terms via min_df and max_df is to improve the performance, not the quality of clusters (as an example).
а так как перфоманс меня вполне себе устраивает, то я с этими параметрами экспериментирова не буду, качество гарантированно не улучшится

### CountVectorizer

In [52]:
vec = CountVectorizer(ngram_range=(3, 3))

X_train_embeddings = vec.fit_transform(X_train)

In [53]:
clf = LinearRegression()
clf.fit(X_train_embeddings, y_train)

y_pred = clf.predict(vec.transform(X_test))
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))

0.5760276435730256


### Биграммы

In [54]:
n_grams = (2, 2)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(vec.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.4991791359819879


### Триграммы

In [55]:
n_grams = (3, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(vec.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.47652328908485386


### би+три граммы

In [56]:
n_grams = (2, 3)

tfidf_vect = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = tfidf_vect.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(tfidf_vect.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.42301859061556984


### Вывод

Векторизация текста с помощью бю и триграмм одновременно дает науилучший результат

### CatBoost

Кэтбуст со встроенным векторайзером

In [57]:
text_features = ['lemma']

In [58]:
X_train_catboost = pd.DataFrame({'lemma': X_train})

In [59]:
model_cb = CatBoostRegressor(learning_rate=0.03,
                            max_depth=10,
                            n_estimators=5000,
                            loss_function='MAE',
                            thread_count=-1, 
                            random_seed=42, 
                            verbose=0, 
                            text_features=text_features,
                            task_type = "GPU" 
                                
)


model_cb.fit(X_train_catboost, y_train, text_features=text_features, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAE is/are not implemented for GPU


In [60]:
X_test_catboost = pd.DataFrame({'lemma': X_test})
 
y_pred = model_cb.predict(X_test_catboost)
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))


0.6458870803821317


#### Вывод

Использование встроенное векторизации кэтбуста не дает хорошего результата

### Кэтбус на фичах TF-IDF

Введем ограничения на частоту слов, иначе ядро не справлялось с убучением

In [61]:
n_grams = (2, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords, min_df=30, max_df=0.93)
X_train_embeddings = vec.fit_transform(X_train)
X_train_embeddings.shape

(255826, 42491)

In [62]:
model_cb = CatBoostRegressor(learning_rate=0.03,
                            max_depth=6,
                            n_estimators=5,
                            loss_function='MAE',
                            thread_count=-1, 
                            random_seed=42, 
                            verbose=0
                                
)


model_cb.fit(X_train_embeddings, y_train,  plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [63]:
X_test_catboost = vec.transform(X_test)
 
y_pred = model_cb.predict(X_test_catboost)
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))

0.9903372578451147


### Попробуем подбор гиперпараметов

Мой компьютер не вывозит гридсёрч, ядро падает, поэтому комменитрую

%%time

model_cb = CatBoostRegressor(thread_count=-1,
                             loss_function='MAE',
                             early_stopping_rounds=10,
                             task_type="GPU")

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}


grid = {'learning_rate': [0.03],
        'depth': [4, 10],
        'l2_leaf_reg': [3, 9]}

grid_search_result = model_cb.grid_search(grid,
                                       cv=3,
                                       X=X_train_embeddings,
                                       y=y_train,
                                       plot=True)

X_test_catboost = vec.transform(X_test)
 
y_pred = model_cb.predict(X_test_catboost)
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))